In [ ]:

AX1="A → (B → A)"
AX2="(A → (B → C)) → ((A → B) → (A → C))"
AX3="(¬B → ¬A) → (A → B)"


from sympy.logic.boolalg import Not, Implies
from sympy import symbols
import re

class RawImplies(Implies):
    @classmethod
    def eval(cls, lhs, rhs):
        # completely disable evaluation
        return None

def remove_outer_brackets(s):
    """
    Removes brackets that encapsulate entire string, if there
    eg: (xyz)-> xyz
    """
    if not s or s[0] != '(' or s[-1] != ')':
        return s  
    depth = 0
    for i, char in enumerate(s):
        if char == '(':
            depth += 1
        elif char == ')':
            depth -= 1
        if depth == 0 and i < len(s) - 1: # not fully wrapped
            return s
    return s[1:-1] # fully wrapped

def parser(s):
    """
    parse an input string of a propositional logic formula 
    to return an array containing each (ordered) component of the formula, 
    where sub-formulae within brackts are treated as one unit.
    """
    if type(s) == list:
        # Case where make sympy function calls parser() on an array of 
        # paresed elements to the right of an implication sign
        # needed since [abc,→,¬,xyz] is correct syntax formula
        s = "".join(f"({item})" for item in s)
    if type(s)!=str:
        print("Parsing error, wrong type entered")
    s=remove_outer_brackets(s)
    s=s.replace(" ", "")
    stack = []
    result = []
    skip=False # for cases where ¬+atomic proposition are paresed together
    for i, char in enumerate(s):
        if skip:
            skip=False
            continue
        if char == '(':
            stack.append(i)
        elif char == ')':
            if stack:
                start = stack.pop()
                # Extract the content inside the current matching parentheses
                if stack==[]:
                    result.append(s[start + 1:i])
        elif stack==[]:
            if char == "¬" and (len(s)>2) and i!=len(s)-1 and s[i+1].isalpha():
                # only parse ¬+atomic proposition together when the two symbols do not compose the entire formula 
                result.append(s[i:i+2])
                skip=True
            else:
                result.append(char)
    return result
    
def make_sympy(s:str):
    """
    Recursive function to turn propositional logic formula from a string 
    to a sympy format
    """
    array=parser(s)
    answer=None
    if len(array)==1 and isinstance(array[0], str) and len(array[0]) == 1:
        char=array[0]
        if not char.isalpha():
            print("Error: lone floating operator",char)
            return
        return symbols(char)
    for i,item in enumerate(array):
        if item == "¬":
            # handle Not operator
            if i==len(array)-1:
                print("Error: floating ¬")
                return
            answer=Not(make_sympy(array[i+1]), evaluate=False)
            # Not operator may not end the parsing, eg: ¬(abc)→(xyz)
        if item == "→":
            if i==0 or i==len(array)-1:
                print("Error floating →")
                return
            if answer:
                answer=RawImplies(answer,make_sympy(array[i+1:]), evaluate=False)
            else:
                answer=RawImplies(make_sympy(array[i-1]),make_sympy(array[i+1:]), evaluate=False)
    # A correctly bracketed equation will not need further evaluation beyond the
	# above recursive calls, thus break the loop 
            break
    return answer

# Initialising the axioms to be used in proof evaluation
axiom1=make_sympy(AX1)
axiom2=make_sympy(AX2)
axiom3=make_sympy(AX3)

In [ ]:
class proof_line:
    def __init__(self,expr,step):
        self.expr=expr
        self.step=step

def check_proof(lines):
    """
    The check_proof() function takes in an array of proof lines 
    (forming a proof) and checks if the given proof is valid or not
    """
    works=True
    if len(lines)<=1: 
        # the first element of the proof always refers to the conclusion
        # So any proof smaller than 2 must be invalid
        print("Missing Proof")
        return
    for i,line in enumerate(lines):
        if i==0:
            continue
        step=line.step.replace(" ","")
        # handling premise introduction:
        if step.lower()=="premise":
            continue # should check against premises needed in proof?
        # handling axiom introductions
        elif step.lower()=="ax1":
            if line.expr != axiom1:
                print("incorrect use of AX1: ",line.expr)
                works=False
                break
        elif step.lower()=="ax2":
            if line.expr != axiom2:
                print("incorrect use of AX2: ",line.expr)
                works=False
                break
        elif step.lower()=="ax3":
            if line.expr != axiom3:
                print("incorrect use of AX3: ",line.expr)
                works=False
                break
        # handling subsitution
        elif re.search(r"sub[0-9]+\[([a-z]:=[a-z¬→\(\),]+)+\]",step.lower()):
            substr=re.search(r"\[([a-zA-Z]:=[a-zA-Z¬→\(\),]+)+\]",step).group(0).replace("[", "").replace("]", "")
            subint=int(re.search(r"[0-9]+",step).group())
            if subint>=i or subint<=0:
                # case of invalid step index reference
                print(f"Incorrect subsitution reference: {line.step} in line {i}.")
                works=False
                break
            items = [item.strip() for item in substr.split(',')]
            subby=lines[subint].expr
            for item in items:
                # loop to find true outcome of substitution step
                thing=re.search(r"([a-zA-Z]):=([a-zA-Z¬→\(\)]+)",item)
                LHS=symbols(thing.group(1))
                RHS=make_sympy(thing.group(2))
                subby=subby.subs(LHS,RHS)
            if subby!=line.expr:
                # checking expression against true outcome
                print(f"Incorrect subsitution:\n{line.step} on {lines[subint].expr} does not give {line.expr}")
                works=False
                print(subby)
                print(line.expr)
                break
        # handling modus ponenes
        elif re.search(r"mp[0-9]+,[0-9]+",step.lower()):
            mpstr=re.search(r"mp([0-9]+),([0-9]+)",step.lower())
            mpint1=int(mpstr.group(1))
            mpint2=int(mpstr.group(2))
            if mpint1>=i or mpint2>=i or mpint1<=0 or mpint2<=0:
                # case of invalid step index reference
                print(f"Incorrect Modus ponens reference: {line.step} in line {i}.")
                works=False
                break
            mp1=lines[mpint1].expr
            mp2=lines[mpint2].expr
            if mp1.func!=RawImplies:
                # case of MP being applied on a non-Implication expression
                print(f"Error, incorrect modus ponnes at {step}")
                works=False
                break
            if mp1.func==RawImplies:
                LHS,RHS=mp1.args
                if LHS==mp2 and RHS==line.expr:
                    # cheking if MP applied correctly
                    works = True
                else:
                    print(f"Incorrect Modes Ponens use:\n{mp1} \nand {mp2} \ndoes not give \n{line.expr}")
                    works=False
                    break
        else: 
            print("Unidentified step: ",step.lower())
            works=False
            break
    if works:
        if lines[0].expr == lines[-1].expr:
            print("VALID PROOF")
        else:
            # final check to see if proof outcome matches target conclusion
            print("Error end statement does not match target")
    else:
        print("INVALID PROOF")


In [3]:
test_proof=[
proof_line(make_sympy("((¬p → q) → (¬p → ¬p))"),"to be proven"),
proof_line(make_sympy("(A → (B → C)) → ((A → B) → (A → C))"),"AX2"),
proof_line(make_sympy("(¬p → (B → C)) → ((¬p → B) → (¬p → C))"),"Sub1[A:=¬p]"),
proof_line(make_sympy("(¬p → (q → ¬p)) → ((¬p → q) → (¬p → ¬p))"),"Sub2[C:=¬p,B:=q]"),
proof_line(make_sympy("(A → (B → A))"),"ax 1"),
proof_line(make_sympy("(¬p → (q → ¬p))"),"Sub4[A:=¬p,B:=q]"),
proof_line(make_sympy("((¬p → q) → (¬p → ¬p))"),"MP3,5")]

check_proof(test_proof)

VALID PROOF


In [5]:
user_input = "((¬p → q) → (¬p → ¬p))"

In [1]:
!pip install ollama

In [4]:
import ollama

client=ollama.Client()

model="Prop_llama"

In [ ]:
response=client.generate(model=model,prompt=user_input)

print(response.response)

1. (A → (B → C)) → ((A → B) → (A → C))...AX2
2. (¬p → (q → ¬p)) → ((¬p → q) → (¬p → ¬p))...Sub1[A:=¬p,C:=¬p,B:=q]
3. (A → (B → A))...AX1
4. (¬p → (q → ¬p))...Sub3[A:=¬p,B:=q]
5. ((¬p → q) → (¬p → ¬p))...MP2,4


In [ ]:
def make_and_check_proof():
    """
    requests an input of a desired proof from the user, 
    calls the LLM to generate said proof, 
    and then checks the proof
    """
    # take input from user:
    target_str=input("Paste your desired formula using →,¬ symbols. Please seperate premises from conclusion using a ⊢: ")
    print(target_str," to be proven")
    # extracts desired conclusion from input
    target_str=re.search(r"⊢([\(\)A-Za-z→¬]+)",target_str.replace(" ","")).group(1)
    target=make_sympy(target_str)
    # calls LLM to generate desired proof
    proof=client.generate(model=model,prompt=target_str)
    print("Generated response:\n",proof.response)
    # processes generated proof so that it can be passed to check_proof fucntion
    proof=(proof.response).splitlines()
    proof_array=[]
    proof_array.append(proof_line(target,"to be proven"))
    for i,line in enumerate(proof):
        line=line.replace(" ","")
        # ignore line if LLM is stating a desired conclusing and not a step
        num=re.search(r"([0-9]+).",line)
        if not(num):
            continue
        # process line to extract expression and step
        stuff=re.search(r"([\(\)A-Za-z→¬]+)...([A-Za-z0-9,\[\]\(\):=→¬]+)",line)
        expr=stuff.group(1)
        expr=make_sympy(expr)
        step=stuff.group(2)
        # add it to the proof_array to form final proof
        proof_array.append(proof_line(expr,step))
    # check proofs validity
    check_proof(proof_array)

# Test cases

1. LLM generated proof testing (5 cases)

In [17]:
make_and_check_proof()

"""
Test case 1: ⊢((¬p → q) → (¬p → ¬p))
"""

⊢((¬p → q) → (¬p → ¬p))  to be proven
Generated response:
 1. (A → (B → C)) → ((A → B) → (A → C))...AX2
2. (¬p → q) → ((¬p → ¬p) → ((¬p → q) → (¬p → ¬p)))...Sub3[A:=¬p,C:=¬p,B:=q]
3. (A → (B → A))...AX1
4. (¬p → (¬p → ¬p))...Sub2[A:=¬p,B:=¬p]
5. ((¬p→q)→(¬p→¬p))...MP3,4
Incorrect subsitution reference: Sub3[A:=¬p,C:=¬p,B:=q] in line 2.
INVALID PROOF


'\nTest case 1: ⊢((¬p → q) → (¬p → ¬p))\n'

In [7]:
make_and_check_proof()

"""
Test case 2: ⊢(p → p)
"""

⊢(p → p)  to be proven
Generated response:
 1. (A → (B → A))...AX1
2. p → (p → p)...Sub3[A:=p,B:=p]
3. p → p...MP1,2
Incorrect subsitution reference: Sub3[A:=p,B:=p] in line 2.
INVALID PROOF


'\nTest case 2: ⊢(p → p)\n'

In [14]:
make_and_check_proof()

"""
Test case 3: ⊢(¬p→(p→q))
"""

⊢(¬p→(p→q))  to be proven
Generated response:
 1. (A → (B → C)) → ((A → B) → (A → C))...AX2
2. (¬p → (p → q)) → ((¬p → p) → (¬p → q))...Sub1[A:=¬p,C:=q,B:=p]
3. (A → (B → A))...AX1
4. (¬p → (p → q))...Sub3[A:=¬p,B:=p]
5. ((¬p→(p→q))→((¬p→p)→(¬p→q)))...MP2,4
Incorrect subsitution:
Sub3[A:=¬p,B:=p] on RawImplies(A, RawImplies(B, A)) does not give RawImplies(~p, RawImplies(p, q))
RawImplies(~p, RawImplies(p, ~p))
RawImplies(~p, RawImplies(p, q))
INVALID PROOF


'\nTest case 3: ⊢(¬p→(p→q))\n'

In [15]:
make_and_check_proof()
# dis one
"""
Test case 4: ⊢(q→(p→q))
"""

⊢(q→(p→q))  to be proven
Generated response:
 1. (A → (B → C)) → ((A → B) → (A → C))...AX2
2. ((q→(p→q))→((q→p)→(q→(p→q))))...Sub1[A:=q,C:=(p→q),B:=p]
3. (A → (B → A))...AX1
4. (q→(p→q))...Sub3[A:=q,B:=p]
5. (q→(p→q))...MP2,4
Incorrect subsitution:
Sub1[A:=q,C:=(p→q),B:=p] on RawImplies(RawImplies(A, RawImplies(B, C)), RawImplies(RawImplies(A, B), RawImplies(A, C))) does not give RawImplies(RawImplies(q, RawImplies(p, q)), RawImplies(RawImplies(q, p), RawImplies(q, RawImplies(p, q))))
RawImplies(RawImplies(q, RawImplies(p, RawImplies(p, q))), RawImplies(RawImplies(q, p), RawImplies(q, RawImplies(p, q))))
RawImplies(RawImplies(q, RawImplies(p, q)), RawImplies(RawImplies(q, p), RawImplies(q, RawImplies(p, q))))
INVALID PROOF


'\nTest case 4: ⊢(q→(p→q))\n'

In [13]:
make_and_check_proof()

"""
Test case 5: p→q,q→r,p ⊢r
"""

 p→q,q→r,p ⊢r  to be proven
Generated response:
 ⊢¬x → (y → z)
1. A → (B → C) → ((A → B) → (A → C))...AX2
2. ¬(x ∧ (y → z)) → ¬(x ∧ ¬(y → z))...Sub3[A:=¬x,C:=¬(y → z),B:¬(x ∧ ¬(y → z))]
Error: lone floating operator ¬
incorrect use of AX2:  RawImplies(A, RawImplies(RawImplies(B, C), RawImplies(RawImplies(A, B), RawImplies(A, C))))
INVALID PROOF


'\nTest case 5: p→q,q→r,p ⊢r\n'

2. Constructed test cases

In [8]:
test_proof=[
proof_line(make_sympy("(q→(p→q))"),""),
proof_line(make_sympy("(A → (B → C)) → ((A → B) → (A → C))"),"AX2"),
proof_line(make_sympy("(¬p → (q → ¬p)) → ((¬p → q) → (¬p → ¬p))"),"Sub1[A:=¬p,C:=¬p,B:=q]"),
proof_line(make_sympy("(A → (B → A))"),"AX1"),
proof_line(make_sympy("(¬p → (q → ¬p))"),"Sub3[A:=¬p,B:=q]"),
proof_line(make_sympy("((¬p → q) → (¬p → ¬p))"),"MP2,4"),
proof_line(make_sympy("(¬p → (p → ¬p)) → ((¬p → p) → (¬p → ¬p))"),"Sub1[A:=¬p,C:=¬p,B:=p]"),
proof_line(make_sympy("(A → (B → A))"),"AX1"),
proof_line(make_sympy("(¬p → (p → ¬p))"),"Sub3[A:=¬p,B:=p]"),
proof_line(make_sympy("(q→(p→q))"),"MP2,8")]

check_proof(test_proof)


Incorrect Modes Ponens use:
RawImplies(RawImplies(~p, RawImplies(q, ~p)), RawImplies(RawImplies(~p, q), RawImplies(~p, ~p))) 
and RawImplies(~p, RawImplies(p, ~p)) 
does not give 
RawImplies(q, RawImplies(p, q))
INVALID PROOF


In [12]:
print("test proof for p→q,q→r,p ⊢r")

test_proof=[
proof_line(make_sympy("r"),"to be proven"),
proof_line(make_sympy("p→q"),"premise"),
proof_line(make_sympy("q→r"),"premise"),
proof_line(make_sympy("p"),"premise"),
proof_line(make_sympy("q"),"MP1,3"),
proof_line(make_sympy("r"),"MP2,4")
]

check_proof(test_proof)

test proof for p→q,q→r,p ⊢r
VALID PROOF


In [44]:
print("test proof for p→q,q→r,p ⊢r")

test_proof=[
proof_line(make_sympy("r"),"to be proven"),
proof_line(make_sympy("(A → (B → C)) → ((A → B) → (A → C))"),"AX2"),
proof_line(make_sympy("(¬p → (B → C)) → ((¬p → B) → (¬p → C))"),"Sub1[A:=¬p]"),
proof_line(make_sympy("(¬p → (q → ¬p)) → ((¬p → q) → (¬p → ¬p))"),"Sub2[C:=¬p,B:=q]"),
proof_line(make_sympy("(A → (B → A))"),"ax 1"),
proof_line(make_sympy("(¬p → (q → ¬p))"),"Sub4[A:=¬p,B:=q]"),
proof_line(make_sympy("((¬p → q) → (¬p → ¬p))"),"MP3,5")]


check_proof(test_proof)

test proof for p→q,q→r,p ⊢r
Error end statement does not match target
